In [ ]:
# Copyright 2025 Forusone

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Agent Development Kit - tavily

### Install adk

In [ ]:
%pip install --upgrade --quiet google-adk \
                               langchain_community \
                               tavily-python

In [ ]:
# @title Project Information

PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"

MODEL_LIST = ["gemini-2.0-flash", \
              "gemini-2.5-pro-preview-03-25", \
              "gemini-2.5-flash-preview-04-17"]

MODEL_NAME = MODEL_LIST[2]
print(MODEL_NAME)

gemini-2.5-flash-preview-04-17


In [ ]:
# @title Authentication to access to GCP
import sys
from pprint import pprint

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id = PROJECT_ID)

## Initialize Vertex AI

* Please refer to configuration to **initialize Vertex AI : [Link](https://google.github.io/adk-docs/get-started/quickstart/#set-up-the-model)**
* You can set up api key rather than usning Vertex AI.


In [ ]:
import os

GOOGLE_GENAI_USE_VERTEXAI = "TRUE"                            #@param {type:"string"}
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = GOOGLE_GENAI_USE_VERTEXAI

GOOGLE_CLOUD_PROJECT = "ai-hangsik"                           #@param {type:"string"}
os.environ["GOOGLE_CLOUD_PROJECT"] = GOOGLE_CLOUD_PROJECT

GOOGLE_CLOUD_LOCATION = "us-central1"                         #@param {type:"string"}
os.environ["GOOGLE_CLOUD_LOCATION"] = GOOGLE_CLOUD_LOCATION

# https://app.tavily.com/home
TAVILY_API_KEY=""     #@param {type:"string"}
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY


## adk with tools

In [ ]:
from google.adk.tools.langchain_tool import LangchainTool
from langchain_community.tools import TavilySearchResults

In [ ]:
# Instantiate the LangChain tool
tavily_tool_instance = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=True,
)

# Wrap it with LangchainTool for ADK
adk_tavily_tool = LangchainTool(tool=tavily_tool_instance)

In [ ]:
#@title Build a Agent with adk.
from google.adk.agents import LlmAgent
from google.adk.tools import google_search
from google.genai import types

PROMPT = """

You are a helpful assistant. Answer user questions using funtion tool when needed.

"""

# https://googleapis.github.io/python-genai/genai.html#genai.types.GenerateContentConfig
generate_content_config= types.GenerateContentConfig(
  temperature=0.5,
  max_output_tokens=1024,
)

# https://google.github.io/adk-docs/api-reference/google-adk.html#google.adk.agents.LlmAgent
agent = LlmAgent(
      name="langchain_tool_agent",
      model=MODEL_NAME,
      description="Agent to answer questions using TavilySearch.",
      instruction="I can answer your questions by searching the internet. Just ask me anything!",
      tools=[adk_tavily_tool] # Add the wrapped tool here
)

In [ ]:
from google.adk.sessions import InMemorySessionService
from google.adk import Agent, Runner

APP_NAME = "tavily_search_app"
USER_ID = "1234"
SESSION_ID = "session1234"

session_service = InMemorySessionService()
session = session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)


In [ ]:
from google.adk.runners import Runner

runner = Runner(agent=agent,
                app_name=session.app_name,
                session_service=session_service)


query = "What is the exchange rate from US dollars to Korean currency today ?"
content = types.Content(role='user', parts=[types.Part(text=query)])

events = runner.run(user_id=session.user_id,
                    session_id=session.id,
                    new_message=content)

for event in events:
    if event.is_final_response():
        final_response = event.content.parts[0].text
        pprint(f"Agent Response: {final_response}\n")
    else:
        pprint("No results")


'No results'
'No results'
('Agent Response: Based on the latest information, as of today, 1 US dollar is '
 'equal to 1,438.36 Korean Won. Please note that exchange rates can fluctuate '
 'throughout the day.\n')


## End of notebook